In [4]:
from fastai2.basics import *
from nbdev.showdoc import *

In [8]:
from captum.attr import IntegratedGradients, NoiseTunnel, visualization as viz
from matplotlib.colors import LinearSegmentedColormap

In [9]:
from fastai2.vision.all import *
path = untar_data(URLs.PETS)/'images'

def is_cat(x): return x[0].isupper()
dls = ImageDataLoaders.from_name_func(
    path, get_image_files(path), valid_pct=0.2, seed=42,
    label_func=is_cat, item_tfms=Resize(128))

learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1)

epoch,train_loss,valid_loss,error_rate,time
0,0.353215,0.156822,0.051421,00:13


epoch,train_loss,valid_loss,error_rate,time
0,0.135744,0.062435,0.025710,00:14


In [10]:
integrated_gradients = IntegratedGradients(learn.model)

In [11]:
paths=list(path.iterdir())
index=random.randint(0,len(paths))
image_path=paths[index]

In [15]:
dl = learn.dls.test_dl([image_path],with_labels=True, bs=1)


enc_inp,enc_preds= dl.one_batch()
decoded_img,decoded_pred=dl.decode((enc_inp,enc_preds))[0][:1],dl.decode((enc_inp,enc_preds))[1][0]

attributions_ig = integrated_gradients.attribute(enc_inp.to(dl.device), target=enc_preds, n_steps=200)

noise_tunnel = NoiseTunnel(integrated_gradients)

attributions_ig_nt = noise_tunnel.attribute(decoded_img.to(dl.device), n_samples=10, nt_type='smoothgrad_sq', target=decoded_pred)
_ = viz.visualize_image_attr_multiple(np.transpose(attributions_ig_nt.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      np.transpose(enc_inp.squeeze().cpu().detach().numpy(), (1,2,0)),
                                      ["original_image", "heat_map"],
                                      ["all", "positive"],
                                      cmap=default_cmap,
                                      show_colorbar=True)

RuntimeError: CUDA out of memory. Tried to allocate 126.00 MiB (GPU 0; 7.93 GiB total capacity; 2.97 GiB already allocated; 38.06 MiB free; 3.09 GiB reserved in total by PyTorch)

Forum Link for discussion: https://discuss.pytorch.org/t/noisetunnel-cuda-out-of-memory-when-trying-out/74494/5

Captum Repo Tutorial: https://github.com/pytorch/captum/blob/master/tutorials/Resnet_TorchVision_Interpret.ipynb